In [4]:
# from sim_to_speed_map.nets.pignn import PowerPIGNN
from utils import read_wind_angles, read_turbine_positions, read_measurement, get_wind_angles_for_range

# import dgl
import torch
import torch.nn as nn
from adamp import AdamP
# from dgl.data.utils import load_graphs
from torch.optim import Adam
from box import Box

import numpy as np

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def get_config():
    nf_dim = 3
    ef_dim = 2
    u_dim = 1

    cfg = Box({
        'model': {
            'edge_in_dim': ef_dim,
            'node_in_dim': nf_dim,
            'global_in_dim': u_dim,
            'n_pign_layers': 3,
            'edge_hidden_dim': 50,
            'node_hidden_dim': 50,
            'global_hidden_dim': 50,
            'residual': True,
            'input_norm': True,
            'pign_mlp_params': {'num_neurons': [256, 128],
                               'hidden_act': 'ReLU',
                               'out_act': 'ReLU'},
            'reg_mlp_params': {'num_neurons': [64, 32, 16],
                               'hidden_act': 'ReLU',
                               'out_act': 'ReLU'},
            'pign_params': {'edge_aggregator': 'mean',
                           'global_node_aggr': 'mean',
                           'global_edge_aggr': 'mean'}
        },
        'train': {
            'batch_size': 512,
            'reset_g_every': 64,
            'num_procs': 6,
            'log_every': 100,
            'train_steps': 20000,
        }
    })
    return cfg

In [5]:
# TODO: import the data

In [ ]:
def main():
    config = get_config()

    # prepare validation data
    val_gs, labels = load_graphs('val_gs3.bin')
    val_us = labels['global_feat']
    val_us = val_us.to(device)
    val_gs = dgl.batch(val_gs).to(device)

    model = PowerPIGNN(**config.model).to(device)
    print(model)

    if 'cuda' in device:
        optimizer = AdamP(model.parameters(), lr=1e-3)
    else:  # for some reason, the entire training code with adamp optimizer broken on cpu
        optimizer = Adam(model.parameters(), lr=1e-3)

    criterion = nn.MSELoss()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=50)

    n_update = 0
    for epoch in range(config.train.train_steps):
        if n_update % config.train.reset_g_every == 0:
            gs, us = prepare_data(config.train.batch_size)
            gs = dgl.batch(gs)
            us = torch.stack(us)

        gs = gs.to(device)
        us = us.to(device)

        nf, ef = gs.ndata['feat'], gs.edata['feat']
        # Augment Euclidean coordinates
        nf = torch.cat([nf, gs.ndata['x'], gs.ndata['y']], dim=-1)

        pred = model(gs, nf, ef, us)
        loss = criterion(pred, gs.ndata['power'])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        learning_rate = optimizer.param_groups[0]['lr']

        n_update += 1

        if n_update % config.train.log_every == 0:
            with torch.no_grad():
                model.eval()
                val_nf, val_ef = val_gs.ndata['feat'], val_gs.edata['feat']
                val_nf = torch.cat([val_nf, val_gs.ndata['x'], val_gs.ndata['y']], dim=-1)
                val_pred = model(val_gs, val_nf, val_ef, val_us)
                val_loss = criterion(val_pred, val_gs.ndata['power'])
                model.train()

            print(f"step {n_update}/{config.train.train_steps}, lr: {learning_rate}, training loss: {loss}, validation loss: {val_loss}")

        # save model pointer
        torch.save(model.state_dict(), f"sim_to_speed_map_{epoch}.pt")